## State of the art for PCR primer melting temperature (tm) calculation

PCR Primer Inspector was a well documented compared to many other online tools. Clicking on the question mark next to each argument displays a page of text regarding the calculation.

The objective of this analysis is to find out if PCR Primer Inspector and biopython (both being modern tools) calculate the same value for Tm of primers.

This comparison was done on July 2024, Biopython version 1.83.

The example case is described below:

The PCR reaction is a version of this [standard](https://www.neb.com/protocols/0001/01/01/taq-dna-polymerase-with-standard-taq-buffer-m0273) protocol:

- 1X Taq Buffer with (NH4)2SO4
- 0.2 mM of each dNTP
- 2 mM MgCl2
- 1 µM Primer I 
- 1 µM Primer II 
- Template DNA

The example buffer has this composition:

[10X Taq Buffer with (NH4)2SO4](https://www.thermofisher.com/order/catalog/product/B33?SID=srch-srp-B33#/B33?SID=srch-srp-B33) (B33) includes:
- 750 mM Tris-HCl (pH 8.8 at 25°C)
- 200 mM (NH4)2SO4
- 0.1% (v/v) Tween 20

This means that the final PCR reaction has: 

- 75 mM Tris-HCl
- 40 mM NH4+ 
- 20 mM SO42-
- 2  mM Mg2+
- 4  mM Cl-


The text below was copied from PCR Primer Inspector

    The total concentration of monovalent cations is expectedly a sum of concentrations of sodium, potassium and ammonium cations 
    (not all of them have to be present in a PCR reaction mix). It is also advised to add the concentration of the dissociated 
    fraction of Tris in the PCR reaction mix (a half of the total Tris concentration may be a good approximation). 
    The Tm prediction may be less accurate in high concentrations (above 100 mM) of monovalent ions capable to form hydrogen bonds 
    (ammonium, Tris) and for bulkier monovalent ions (tetramethylammonium).

Hence, "Cations" concentration should be 75/2 (half of Tris-HCl) + 40 (NH4+) = 77.5 mM.
There should be very little of other monovalent cations.

The list of arguments is:

- Primer concentration (nM): 1000 
- Total monovalent cation concentration (mM): 77.5 
- Mg2+concentration (mM): 2 
- Total dNTPs concentration (mM): 0.8

This values above were added to the input form of PCR Primer Inspector:

![](page1.png)

The result is 54.4°C for "Owczarzy 2008" and 54.9°C for "von Ahsen 2001". The use of these  
references is described below. The text below was copied from PCR Primer Inspector:

    MELTING TEMPERATURE CALCULATION

    All the melting temperature calculations provided here are based on the 
    nearest neighbor method with parameters described in references [1] and [4]. 

    Results of two different algorithms (references [2] and [3]) for monovalent 
    and magnesium cation correction are offered.

    1.  A unified view of polymer, dumbbell, and oligonucleotide DNA nearest-neighbor thermodynamics
        SantaLucia J. Jr. Proc Natl Acad Sci U S A. 1998;95(4):1460-5.

    2.  Oligonucleotide melting temperatures under PCR conditions: nearest-neighbor corrections for Mg(2+), 
        deoxynucleotide triphosphate, and dimethyl sulfoxide concentrations with comparison to alternative 
        empirical formulas
        von Ahsen N., Wittwer C. T., Schütz E.
        Clin Chem. 2001;47(11):1956-61.

    3.  Predicting stability of DNA duplexes in solutions containing magnesium and monovalent cations
        Owczarzy R., Moreira B. G., You Y., Behlke M. A., Walder J. A.
        Biochemistry. 2008;47(19):5336-53.

    4.  The thermodynamics of DNA structural motifs
        SantaLucia J. Jr., Hicks D.
        Annu Rev Biophys Biomol Struct. 2004;33:415-40.

The first statement is somewhat ambiguous as the data in 1 and 4 are different (see below) although indeed very similar.

#### SantaLucia 1998
![](sl_1998_table2.png)

#### SantaLucia 2004
![](sl_and_hicks_2004_table1.png)

As can be observed by comparing the two tables above, they only differ for the AA/TT entry
and for Initiation / Init. w term. GC and Terminal AT penalty / Init. w term. AT.


### Biopython

From Biopython [docs](https://biopython.org/DIST/docs/api/Bio.SeqUtils.MeltingTemp-module.html):

table: Thermodynamic NN values, eight tables are implemented: For DNA/DNA hybridizations:

- DNA_NN1: values from Breslauer et al. (1986)
- DNA_NN2: values from Sugimoto et al. (1996)
- DNA_NN3: values from Allawi & SantaLucia (1997) (default) <===<<<< Should be the same as SantaLucia 1998
- DNA_NN4: values from SantaLucia & Hicks (2004)                   

The salt correction method from Owczarzy et al. (2008) was chosen since this was what was used for PCR Primer Inspector.

[7](https://biopython.org/DIST/docs/api/Bio.SeqUtils.MeltingTemp-module.html#salt_correction) = Complex formula with decision tree and 7 empirical constants. Mg2+ is corrected for dNTPs binding (if present) (Owczarzy et al. (2008), Biochemistry 47: 5336-5353)

In [5]:
import Bio
Bio.__version__

'1.83'

In [6]:
from Bio.SeqUtils import MeltingTemp as mt

The primer sequence is identical to the one used for the PCR Primer Inspector

In [7]:
print( mt.Tm_NN("ATGGCAGTTGAGAAGA", 
       nn_table=mt.DNA_NN3,
       Na=40,
       Tris=75.0,
       Mg=2.0, 
       dnac1=1000, 
       dnac2=0, 
       dNTPs=0.8, 
       saltcorr=7) )

57.02281745439029


### The biopython module reports 57.0228 °C while the PCR Primer Inspector reported 54.4°C.
The difference is about 3 °C.

This is a greater difference that I expected with care taken to use the same thermodynamic data. 
I contacted Vladimir Cermak by email:

    Hi, I have tried to replicate the results from PCR Primer Inspector@molbiotools.com
    with biopython using the same parameters and the same data as far as I could tell. I still have a difference of about 3C.
    
    I compiled the comparison in this notebook:
    
    https://github.com/BjornFJohansson/tm/blob/master/tm.ipynb
    
    Would you care to have a look at it? to see if there is something obviously wrong?
    Is the code used by PCR Primer Inspector available for comparison?
    
    All the best,
    Björn Johansson

Response:

    I tried to figure this out, but as I didn't see the python code I can only guess.
    
    PCR Primer Inspector uses the parameters from [1], but changing them to [4] gives only 0.1°C difference.
    
    What is probably the source of the discordant results is different understanding and implementation of this (from SantaLucia 1998):
    
    "For self-complementary oligonucleotide duplexes, the Tm is calculated from the predicted dH° and dS° and the total oligonucleotide 
    strand concentration CT, by using the equation ...[3]... For non-selfcomplementary molecules, CT in Eq. 3 is replaced by CT/4 
    if the strands are in equal concentration or by (CA - CB/2) if the strands are at different concentrations, where CA and CB are 
    the concentrations of the more concentrated and less concentrated strands, respectively."
        
    That sounds like it should be CT for PCR conditions where C(primer)>>C(template) and CT/4 for an equimolar mix of complementary oligos. 
    However, when checking the available code of Primer3, I found they use CT/4 for PCR conditions. 
    I decided to be compliant with Primer3 (also Primer BLAST).
    
    If I change the algorithm to CT instead of CT/4, it gives 57.0 for half the Tris concentration and 
    57.9 for the whole Tris counted as monovalent cations.
    
    So it may be that there is a mistake in Primer3 (and hence also in PCR Primer Inspector), 
    which is a kind of "gold standard" in the field....
    
    Best regards,
    Vladimir Cermak


This means that the PCR Primer Inspector, Primer3 and other widely used tools implements the primer concentration
in a way not intended by the original author. The question is how to correctly emulate this using Biopython.
Fortunately the Biopython documentation explains this nicely:

[![](dnac1_2.png)](https://biopython.org/DIST/docs/api/Bio.SeqUtils.MeltingTemp-pysrc.html#Tm_NN)

If I instead assign dnac1 and dnac2 to half of the total primer concentration according to the biopyton docs,
the result become rather similar.

In [8]:
from Bio.SeqUtils import MeltingTemp as mt
print( mt.Tm_NN("ATGGCAGTTGAGAAGA", 
       nn_table=mt.DNA_NN3,
       Na=40,
       Tris=75.0,
       Mg=2.0, 
       dnac1=1000/2, 
       dnac2=1000/2, 
       dNTPs=4*0.2, 
       saltcorr=7) )

54.49960170791479


PCR Primer Inspector gave 54.4 while biopython above gave 54.49 C or a 0.09 C higher tm.

In [9]:
from Bio.SeqUtils import MeltingTemp as mt
print( mt.Tm_NN("ATGGCAGTTGAGAAGA", 
       nn_table=mt.DNA_NN4,
       Na=40,
       Tris=75.0,
       Mg=2.0, 
       dnac1=1000/2, 
       dnac2=1000/2, 
       dNTPs=0.8, 
       saltcorr=7) )

54.42261614376554


With the SantaLucia 2004 data, the results agree completely.